In [1]:
import os 
from dotenv import load_dotenv

load_dotenv()

try:
    GOOGLE_API_KEY= os.getenv("GOOGLE_API_KEY")
    if(GOOGLE_API_KEY):
        print("API Key loaded successfully")
    else:
        print("API Key not found")
except Exception as e:
    print(f"Error loading API Key: {e}")





API Key loaded successfully


In [5]:
from google.genai import types
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search, AgentTool, ToolContext
from google.adk.code_executors import BuiltInCodeExecutor

print("✅ Google ADK imported successfully.")

✅ Google ADK imported successfully.


In [6]:
def show_python_code_and_result(response):
    for i in range(len(response)):
        # Check if the response contains a valid function call result from the code executor
        if (
            (response[i].content.parts)
            and (response[i].content.parts[0])
            and (response[i].content.parts[0].function_response)
            and (response[i].content.parts[0].function_response.response)
        ):
            response_code = response[i].content.parts[0].function_response.response
            if "result" in response_code and response_code["result"] != "```":
                if "tool_code" in response_code["result"]:
                    print(
                        "Generated Python Code >> ",
                        response_code["result"].replace("tool_code", ""),
                    )
                else:
                    print("Generated Python Response >> ", response_code["result"])


print("✅ Helper functions defined.")

✅ Helper functions defined.


In [7]:
retry_config = types.HttpRetryOptions(
    attempts = 5,
    exp_base = 7,
    initial_delay = 1,
    http_status_codes = [429, 500, 503, 504]
)

# 1.1 Currency Converter System

In [8]:
def get_fee_for_payment_method(method: str) -> dict:
    """Looks up the transaction fee percentage for a given payment method.

    This tool simulates looking up a company's internal fee structure based on
    the name of the payment method provided by the user.

    Args:
        method: The name of the payment method. It should be descriptive,
                e.g., "platinum credit card" or "bank transfer".

    Returns:
        Dictionary with status and fee information.
        Success: {"status": "success", "fee_percentage": 0.02}
        Error: {"status": "error", "error_message": "Payment method not found"}
    """

    fee_db = {
        "platinum credit card" : 0.02,
        "gold credit card" : 0.035,
        "silver credit card" : 0.05,
        "bank transfer" : 0.01,
        "paypal" : 0.025,
        "apple pay" : 0.03,
        "google pay" : 0.03,
        "amazon pay" : 0.035,
        "shopify pay" : 0.04,
    }

    fee = fee_db.get(method.lower())
    if fee is not None:
        return {"status": "success", "fee_percentage": fee}
    else:
        return {"status":"error", "error_message": f"Payment method '{method}' not found in the database."}
    
print("fee lookup tool defined.")



fee lookup tool defined.


In [9]:
def get_exchange_rate(base_currency: str, target_currency: str) -> dict:
    """Looks up and returns the exchange rate between two currencies.

    Args:
        base_currency: The ISO 4217 currency code of the currency you
                       are converting from (e.g., "USD").
        target_currency: The ISO 4217 currency code of the currency you
                         are converting to (e.g., "EUR").

    Returns:
        Dictionary with status and rate information.
        Success: {"status": "success", "rate": 0.93}
        Error: {"status": "error", "error_message": "Unsupported currency pair"}
    """
    rate_db = {
        "usd":{
            "eur": 0.93,
            "gbp": 0.82,
            "inr": 83.21,
        },
        "eur":{
            "usd": 1.07,
            "gbp": 0.89,
            "inr": 85.47,
        }   
    }

    base = base_currency.lower()
    target = target_currency.lower()

    rate = rate_db.get(base, {}).get(target)

    if rate is not None:
        return {"status": "success", "rate": rate}
    else:
        return {"status": "error", "error_message": f"Unsupported currency pair: {base_currency} to {target_currency}"}

print("exchange rate tool defined.")

exchange rate tool defined.


In [10]:
# Currency agent with custom function tools
currency_agent = LlmAgent(
    name="currency_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""You are a smart currency conversion assistant.

    For currency conversion requests:
    1. Use `get_fee_for_payment_method()` to find transaction fees
    2. Use `get_exchange_rate()` to get currency conversion rates
    3. Check the "status" field in each tool's response for errors
    4. Calculate the final amount after fees based on the output from `get_fee_for_payment_method` and `get_exchange_rate` methods and provide a clear breakdown.
    5. First, state the final converted amount.
        Then, explain how you got that result by showing the intermediate amounts. Your explanation must include: the fee percentage and its
        value in the original currency, the amount remaining after the fee, and the exchange rate used for the final conversion.

    If any tool returns status "error", explain the issue to the user clearly.
    """,
    tools=[get_fee_for_payment_method, get_exchange_rate],
)

print("✅ Currency agent created with custom function tools")
print("🔧 Available tools:")
print("  • get_fee_for_payment_method - Looks up company fee structure")
print("  • get_exchange_rate - Gets current exchange rates")

✅ Currency agent created with custom function tools
🔧 Available tools:
  • get_fee_for_payment_method - Looks up company fee structure
  • get_exchange_rate - Gets current exchange rates


In [11]:
currency_runner = InMemoryRunner(agent=currency_agent)
_ = await currency_runner.run_debug(
    "I want to convert 500 US Dollars to Euros using my Platinum Credit Card. How much will I receive?"
)

App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "C:\Users\aarya\Desktop\Aaryan\Projects\Agents\ai-agents-workshop\agentenv\Lib\site-packages\google\adk\agents", which implies app name "agents".



 ### Created new session: debug_session_id

User > I want to convert 500 US Dollars to Euros using my Platinum Credit Card. How much will I receive?


currency_agent > You will receive 455.70 Euros.

This is based on the following:
*   **Fee:** A 2.00% fee is applied for using a platinum credit card, which amounts to 10.00 USD.
*   **Amount after fee:** 490.00 USD remains after the fee.
*   **Exchange rate:** The exchange rate from USD to EUR is 0.93.
*   **Final amount:** 490.00 USD is converted to 455.70 EUR.


# 1.2 Enhanced Currency Converter System
using Built in code Executor, this is better in performing mathematical calculations

In [12]:
calculation_agent = LlmAgent(
    name = "CalculationAgent",
    model = Gemini(model = "gemini-2.5-flash-lite", retry_options = retry_config),
     instruction="""You are a specialized calculator that ONLY responds with Python code. You are forbidden from providing any text, explanations, or conversational responses.
 
     Your task is to take a request for a calculation and translate it into a single block of Python code that calculates the answer.
     
     **RULES:**
    1.  Your output MUST be ONLY a Python code block.
    2.  Do NOT write any text before or after the code block.
    3.  The Python code MUST calculate the result.
    4.  The Python code MUST print the final result to stdout.
    5.  You are PROHIBITED from performing the calculation yourself. Your only job is to generate the code that will perform the calculation.
   
    Failure to follow these rules will result in an error. """,
    code_executor = BuiltInCodeExecutor()
)

In [17]:
enhanced_currency_agent = LlmAgent(
    name="enhanced_currency_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    # Updated instruction
    instruction="""You are a smart currency conversion assistant. You must strictly follow these steps and use the available tools.

  For any currency conversion request:

   1. Get Transaction Fee: Use the get_fee_for_payment_method() tool to determine the transaction fee.
   2. Get Exchange Rate: Use the get_exchange_rate() tool to get the currency conversion rate.
   3. Error Check: After each tool call, you must check the "status" field in the response. If the status is "error", you must stop and clearly explain the issue to the user.
   4. Calculate Final Amount (CRITICAL): You are strictly prohibited from performing any arithmetic calculations yourself. You must use the calculation_agent tool to generate Python code that calculates the final converted amount. This 
      code will use the fee information from step 1 and the exchange rate from step 2.
   5. Provide Detailed Breakdown: In your summary, you must:
       * State the final converted amount.
       * Explain how the result was calculated, including:
           * The fee percentage and the fee amount in the original currency.
           * The amount remaining after deducting the fee.
           * The exchange rate applied.
    """,
    tools=[
        get_fee_for_payment_method,
        get_exchange_rate,
        AgentTool(agent=calculation_agent),  
    ],
)

print("✅ Enhanced currency agent created")
print("🎯 New capability: Delegates calculations to specialist agent")
print("🔧 Tool types used:")
print("  • Function Tools (fees, rates)")
print("  • Agent Tool (calculation specialist)")

✅ Enhanced currency agent created
🎯 New capability: Delegates calculations to specialist agent
🔧 Tool types used:
  • Function Tools (fees, rates)
  • Agent Tool (calculation specialist)


In [24]:
enhanced_runner = InMemoryRunner(agent=enhanced_currency_agent, app_name = "enhanced_currency_agent")

App name mismatch detected. The runner is configured with app name "enhanced_currency_agent", but the root agent was loaded from "C:\Users\aarya\Desktop\Aaryan\Projects\Agents\ai-agents-workshop\agentenv\Lib\site-packages\google\adk\agents", which implies app name "agents".


In [25]:
# Test the enhanced agent
response = await enhanced_runner.run_debug(
    "Convert 1,250 USD to INR using a Bank Transfer. Show me the precise calculation.",
    verbose = True,
    session_id = "new_debug_session",
)


 ### Created new session: new_debug_session

User > Convert 1,250 USD to INR using a Bank Transfer. Show me the precise calculation.


enhanced_currency_agent > [Calling tool: get_fee_for_payment_method({'method': 'Bank Transfer'})]
enhanced_currency_agent > [Tool result: {'status': 'success', 'fee_percentage': 0.01}]


enhanced_currency_agent > [Calling tool: get_exchange_rate({'base_currency': 'USD', 'target_currency': 'INR'})]
enhanced_currency_agent > [Tool result: {'status': 'success', 'rate': 83.21}]


App name mismatch detected. The runner is configured with app name "enhanced_currency_agent", but the root agent was loaded from "C:\Users\aarya\Desktop\Aaryan\Projects\Agents\ai-agents-workshop\agentenv\Lib\site-packages\google\adk\agents", which implies app name "agents".


enhanced_currency_agent > [Calling tool: CalculationAgent({'request': 'Calculate the conversion of 1250 USD ...)]
enhanced_currency_agent > [Tool result: {'result': 'tool_code\nusd_amount = 1250\nexchange_rate = 83.21\nfee_percentage = 0.01\n\nfee_amount...]


App name mismatch detected. The runner is configured with app name "enhanced_currency_agent", but the root agent was loaded from "C:\Users\aarya\Desktop\Aaryan\Projects\Agents\ai-agents-workshop\agentenv\Lib\site-packages\google\adk\agents", which implies app name "agents".


enhanced_currency_agent > [Calling tool: CalculationAgent({'request': 'tool_code\nusd_amount = 1250\nexchang...)]
enhanced_currency_agent > [Tool result: {'result': 'code_output\ninr_amount_after_fee=103961.25\n'}]
enhanced_currency_agent > To convert 1,250 USD to INR using Bank Transfer, here's the breakdown of the calculation:

1.  **Transaction Fee**: A fee of 1% is applied to the transaction. This amounts to 12.50 USD.
2.  **Amount After Fee**: After deducting the fee, 1,237.50 USD remains (1,250 USD - 12.50 USD).
3.  **Exchange Rate**: The exchange rate used is 83.21 INR per USD.
4.  **Final Converted Amount**: The remaining 1,237.50 USD is converted to INR, resulting in **103,961.25 INR**.


In [18]:
show_python_code_and_result(response)

Generated Python Response >>  ```python
amount_usd = 1250
fee_percentage = 0.01
exchange_rate = 83.21

fee_amount_usd = amount_usd * fee_percentage
amount_after_fee_usd = amount_usd - fee_amount_usd
amount_in_inr = amount_after_fee_usd * exchange_rate

print(f"Fee percentage: {fee_percentage*100}%")
print(f"Fee amount in USD: {fee_amount_usd:.2f}")
print(f"Amount after fee in USD: {amount_after_fee_usd:.2f}")
print(f"Exchange rate: {exchange_rate}")
print(f"Final amount in INR: {amount_in_inr:.2f}")
```
